<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>

## Introduction

In this notebook we will explore the neighborhoods in Toronto Canada using the Fourdquare API and the K-means clustering method. The goal is to scrape a webpage for data, convert it into a pandas data frame, clean and wrangle the data for use in a K-meansw clustering algorithm, and then analyze the makeup of the neighbourhood clusters in Toronto, Canada. We want to see if there is any noticeable defnining characteristics of how neighbourhoods in Toronto, Canada are layed out.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    
<font size = 3>

1. <a href="#item1">Scrape Webpage for Data and Clean it</a>

2. <a href="#item2">Get Postal Code Geogrpahic Coordinates and add to Dataframe.</a>

3. <a href="#item3">Exploring the Neighborhoods in Toronto</a>

First, lets download all the dependencies and libraries we will need. 

In [1]:
#Library for parsing html page
from bs4 import BeautifulSoup

#library for sending and recieveing HTTP/1.1 requests
import requests

#Library to handle JSON files
import json

#Library for data analysis, also set options so that we can display entire dataframes
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)\

#Library to handle vectorized data
import numpy as np

#geocoder library for getting location's geographical coordiantes
from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Scrape Webpage for Data and Clean it

In this section we will scrape the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, for any tables on the web page. There should only be the one containing all the neighbourhoods, boroughs, and postal codes assocaiated with Toronto.

Our final cleaned dataframe should have the following attributes
    1. The dataframe constist of three columns: PostalCode, Borough, and Neighbourhood
    2. We will ignore all cells that have a Borough of 'Not assigned'. These rows will be dropped form the dataframe.
    3. Any duplicate PostalCodes will be combined into a single row taking the form... "M5A | Downtown Toronto | Regent Park, Harbourfront. 
    4. If a cell has a borough but a Not assigned neighbourhood, then the neighboorhood will be the same as the borough.

Lets start by scraping the Wikipedia page for Toronta postal codes and turning that data we find into a pandas dataframe.

In [2]:
#Set url to scrape and get contents of that url using requests library.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'   
response = requests.get(url)

#Use beautiful soup library to parse html document and look for table tags.
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')

#convert table found in html document to pandas data frame.
df_tor = pd.read_html(str(table), flavor='bs4')[0]

In [3]:
#display a preview of the scraped data frame
df_tor.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Lets get an idea of the type and shape of data we are dealing with.

In [4]:
#Get the type of each column in the data frame
df_tor.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

In [5]:
#get the shape of the data frame
print("The data frame consists of {} rows, and {} columns.".format(df_tor.shape[0], df_tor.shape[1]))

The data frame consists of 180 rows, and 3 columns.


In [6]:
#Get the occurence of each name in the Boroughs column.
df_tor['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

We are only wanting to process cells with an assigned borough so we will drop all rows from the dataframe where borough = 'Not assigned'. We know from the value_counts() function above there are 77 such rows.

In [7]:
#drop all rows where borough = "not assigned"
df_tor.drop(df_tor[df_tor['Borough'] == 'Not assigned'].index, inplace = True)

#reset the index after dropping rows
df_tor.reset_index(drop=True, inplace=True)
df_tor.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
#check if any neighbourhoods have the "Not assigned" value
df_tor['Neighbourhood'].str.contains('Not assigned').any()

False

In [9]:
#Check how many rows we have after dropping (should be 180 - 77 = 103)
print("There are now {} rows and {} columns in the data frame".format(df_tor.shape[0], df_tor.shape[1]))

There are now 103 rows and 3 columns in the data frame


We now have a clean dataframe of all the postal codes and assocatied boroughs and neighborhoods in Toronto. In the next section, we will prepare the dataframe for use with the Foursquare API. To do this, we need to get the latitude and longitude coordinates of each neighborhood. 

## 2. Get Postal Code Geogrpahic Coordinates and add to Dataframe.

In this section we will obtain the latitude and longitude coordinates for all the Toronta zipcodes in our dataframe using the geopy library and the Naminatim API.

In [10]:
#Define latitude and longitude lists to be obatined and the user_agent for the API.
geolocator = Nominatim(user_agent="Toronto_explorer")
latitude = []
longitude = []

#Obtain postal codes for each postal code in Toronta, Canada and append them to the previously defined lists.
for postal_code in df_tor['Postal Code']:
    try:
        location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(None)
        longitude.append(None)
    
#add the Latitude and Longitude lists to the data frame
df_tor['Latitude'] = latitude
df_tor['Longitude'] = longitude

#show the dataframe
df_tor

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.653482,-79.383935
7,M3B,North York,Don Mills,NaN,NaN
8,M4B,East York,"Parkview Hill, Woodbine Gardens",NaN,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson",NaN,NaN


Unfortunately, I am not able to get the geopy library to return the latitude and longitude values for every zipcode in Toronto, so for now I will use the supplied csv file http://cocl.us/Geospatial_data to fill in the missing coordinates. 

In [11]:
#Import Geospatiol_data csv from link above as a pandas dataframe.
df_coords = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

#display first 5 rows of dataframe
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#drop Latitude and Longitude columns from the df_tor dataframe before we merge 
df_tor.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

#merge the two data frames
toronto_data = pd.merge(df_tor, df_coords, on ='Postal Code')

#show first 5 rows of merged dataframe
toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


We now have the completed dataframe to work with. We will use this data frame going foward for all analysis.

## 3. Exploring and Clustering the Neighborhoods in Toronto

In this section we will explore and cluster the neighbourhoods in Toronto.

First we will creat a map of Toronto canada and plot each one of our boroughs on it.

In [13]:
#get coordinates for Toronto, Canada.
geolocator = Nominatim(user_agent="toronto_explorer")
loc = geolocator.geocode('Toronto, Ontario')

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[loc.latitude, loc.longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them. Lets get the top 100 venues in each neighborhood that are in a radius of 1500 meters (which after some experimentation is the rquired distance so that all neighbourhoods will return at least one venue in the area).

Note: I have removed my credentials for security purposed but you could imput your own credentials and get the same results for the toronto area.

In [14]:
#Define foursquare credentials, version and parameters for exploration
CLIENT_ID =
CLIENT_SECRET =
VERSION = '20180605'
Limit = 100
radius = 1500

#### Explore Neighbourhoods in Toronto

We now need to query the Foursquare API to obtain the top 100 venues for each neighbourhood in our dataframe. Lets create a function that gets the relevant information for nearby venues in each borough. In this case we want to know the venue name, geogrpahic coordinates and category.

In [15]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we use the above function on the toronto_data data frame to generate a second data frame of toronto venues.

In [16]:
#get data from fourswaure API and creat new dataframe.
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes=toronto_data['Latitude'], longitudes=toronto_data['Longitude']) 

#Size of venues dataframe
print('Dataframe consists of {} rows and {} columns.'.format(toronto_venues.shape[0], toronto_venues.shape[1]))

toronto_venues.head(10)

Dataframe consists of 6890 rows and 7 columns.


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
5,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
6,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
7,Parkwoods,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop
8,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
9,Parkwoods,43.753259,-79.329656,Food Basics,43.760549,-79.326045,Supermarket


Lets find out how many venues were returned for each neighbourhood based on our request parameters submitted to the Foursquare API.

In [17]:
#counts of venues in each neighborhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,62,62,62,62,62,62
"Alderwood, Long Branch",44,44,44,44,44,44
"Bathurst Manor, Wilson Heights, Downsview North",41,41,41,41,41,41
Bayview Village,16,16,16,16,16,16
"Bedford Park, Lawrence Manor East",81,81,81,81,81,81
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",12,12,12,12,12,12
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


Now that we have created a dataframe for each Toronoto neighborhoods lets go ahead and analyze each neighborhood.

#### Analyze Each Neighbourhood
First we will do some onehot encoding before we apply the models. This will allow us to convert our categorical variables into a form that allows better prediction when provided to a ML algorithm. A 1 means the venue type is present in the neighbourhood while a 0 indicates there is no suh venues within the search radius we supplied to the Foursquare API.

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print('onehot dataframe consists of {} rows and {} columns.'.format(toronto_onehot.shape[0], toronto_onehot.shape[1]))
toronto_onehot.head()

onehot dataframe consists of 6890 rows and 350 columns.


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salo

Now let's create a new dataframe called toronto_grouped by grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category.

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print('grouped dataframe consists of {} rows and {} columns.'.format(toronto_grouped.shape[0], toronto_grouped.shape[1]))
toronto_grouped.head(10)

grouped dataframe consists of 99 rows and 350 columns.


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Salad Place,Salo

Now we can find the top 5 most common venues in each neighbourhood.

In [20]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.16
1         Shopping Mall  0.05
2                Bakery  0.03
3  Gym / Fitness Center  0.03
4      Asian Restaurant  0.03


----Alderwood, Long Branch----
              venue  freq
0       Pizza Place  0.07
1              Café  0.05
2              Bank  0.05
3     Grocery Store  0.05
4  Toy / Game Store  0.05


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Park  0.15
1  Coffee Shop  0.07
2  Pizza Place  0.07
3         Bank  0.05
4  Gas Station  0.05


----Bayview Village----
         venue  freq
0         Bank  0.12
1         Park  0.12
2  Gas Station  0.12
3        Trail  0.12
4         Café  0.06


----Bedford Park, Lawrence Manor East----
                venue  freq
0              Bakery  0.09
1  Italian Restaurant  0.07
2         Coffee Shop  0.06
3    Sushi Restaurant  0.06
4          Bagel Shop  0.05


----Berczy Park----
                 venue  freq
0          Cof

----India Bazaar, The Beaches West----
               venue  freq
0               Park  0.06
1        Coffee Shop  0.06
2  Indian Restaurant  0.05
3            Brewery  0.04
4               Café  0.04


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy  0.18
1           Bank  0.12
2  Grocery Store  0.12
3    Supermarket  0.06
4         Bakery  0.06


----Kennedy Park, Ionview, East Birchmount Park----
                  venue  freq
0  Fast Food Restaurant  0.10
1           Coffee Shop  0.10
2        Sandwich Place  0.08
3              Pharmacy  0.08
4    Chinese Restaurant  0.08


----Kensington Market, Chinatown, Grange Park----
                  venue  freq
0                   Bar  0.06
1           Coffee Shop  0.05
2                  Café  0.05
3  Caribbean Restaurant  0.04
4          Cocktail Bar  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
            venue  freq
0     Coffee Shop  0.12
1  Sandwich Pla

           venue  freq
0           Bank  0.12
1    Pizza Place  0.09
2    Coffee Shop  0.09
3  Grocery Store  0.06
4    Supermarket  0.03


----Weston----
                   venue  freq
0   Fast Food Restaurant  0.04
1  Vietnamese Restaurant  0.04
2      Convenience Store  0.04
3         Sandwich Place  0.04
4          Train Station  0.04


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern Restaurant  0.08
1                Pizza Place  0.06
2           Asian Restaurant  0.06
3             Sandwich Place  0.04
4               Burger Joint  0.04


----Willowdale, Newtonbrook----
               venue  freq
0  Korean Restaurant  0.14
1        Coffee Shop  0.11
2               Café  0.07
3    Bubble Tea Shop  0.05
4         Restaurant  0.05


----Willowdale, Willowdale East----
                 venue  freq
0    Korean Restaurant  0.10
1          Coffee Shop  0.07
2      Bubble Tea Shop  0.07
3  Japanese Restaurant  0.05
4        Grocery Store  0.05


----Willowd

Now let's we want to opbtain the top 1o venues from each neighborhood and put them in a data frame.

In [21]:
#Define function to sort values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Use the above function to create the new data frame with the top 10 venues sorted from most common to least common.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Asian Restaurant,Gym / Fitness Center,Coffee Shop,Cantonese Restaurant,Caribbean Restaurant,Bakery,Department Store,Breakfast Spot
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Burger Joint,Toy / Game Store,Park,Café,Bank,Grocery Store,Bakery,Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Pizza Place,Coffee Shop,Gas Station,Bank,Bridal Shop,Men's Store,Mediterranean Restaurant,Shopping Mall,Supermarket
3,Bayview Village,Gas Station,Park,Trail,Bank,Restaurant,Shopping Mall,Café,Chinese Restaurant,Grocery Store,Athletics & Sports
4,"Bedford Park, Lawrence Manor East",Bakery,Italian Restaurant,Sushi Restaurant,Coffee Shop,Bagel Shop,Pizza Place,Pub,Café,Ice Cream Shop,Restaurant
5,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Park,Beer Bar,Pizza Place,Farmers Market,Italian Restaurant
6,"Birch Cliff, Cliffside West",Convenience Store,Bus Stop,Golf Course,General Entertainment,Gym,Café,Park,Diner,Restaurant,Ice Cream Shop
7,"Brockton, Parkdale Village, Exhibition Place",Café,Gift Shop,Bakery,Coffee Shop,Restaurant,Furniture / Home Store,Bar,Soccer Stadium,Diner,Theater
8,"Business reply mail Processing Centre, South C...",Coffee Shop,Brewery,Indian Restaurant,Bakery,Café,Park,Italian Restaurant,Grocery Store,Beach,Sushi Restaurant
9,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Café,Coffee Shop,Gym,Restaurant,Harbor / Marina,Hotel,Brewery,Boat or Ferry,Pizza Place


Now that we have exlpored our data lets cluster the neighborhoods into 5 clusters using K-means clustering.

#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters. This will organize the neighbourhoods into clusters based on the most common venue types in each neighbourhood.

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 3,
       1, 3, 2, 1, 1, 3, 3, 1, 1, 1, 3, 3, 2, 1, 1, 3, 2, 1, 1, 1, 3, 3,
       1, 3, 1, 1, 1, 1, 0, 3, 1, 1, 2, 3, 1, 1, 3, 1, 1, 2, 3, 1, 1, 1,
       1, 1, 2, 1, 3, 3, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 3,
       2, 3, 2, 2, 1, 1, 3, 3, 3, 3, 3])

Let's create a new dataframe that includes the assigned cluster as well as the top 10 venues for each neighborhood.

In [24]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

#convert cluster labels column to integetrs
#oronto_merged['Cluster Labels'] = tornto_merged['Cluster Labels'].astype(int)

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Pharmacy,Intersection,Coffee Shop,Bank,Supermarket,Gas Station,Park,Medical Supply Store,Caribbean Restaurant,Office
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Middle Eastern Restaurant,Fast Food Restaurant,Gym,Park,Shoe Store,Gym / Fitness Center,Grocery Store,History Museum,Bank
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Café,Restaurant,Park,Pub,Thai Restaurant,Farmers Market,Gastropub,Food Truck,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Restaurant,Sandwich Place,Dessert Shop,Vietnamese Restaurant,Food Court,Cosmetics Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Café,Japanese Restaurant,Park,Ramen Restaurant,Gastropub,Thai Restaurant,Pizza Place,Restaurant,Italian Restaurant
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,3,Pharmacy,Grocery Store,Bank,Playground,Supermarket,Golf Course,Café,Liquor Store,Shopping Mall,Skating Rink
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Gas Station,Caribbean Restaurant,Bank,Supermarket,Chinese Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Coffee Shop,Restaurant,Japanese Restaurant,Bank,Pizza Place,Italian Restaurant,Middle Eastern Restaurant,Park,Sandwich Place,Gym
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,Pharmacy,Pizza Place,Gym / Fitness Center,Park,Fast Food Restaurant,Athletics & Sports,Intersection,Brewery,Gastropub,Mexican Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Café,Restaurant,American Restaurant,Theater,Diner,Pizza Place


Lets plot the resulting clusters on the map of toronto to visualize our clusters.

In [25]:
# create map
map_clusters = folium.Map(location=[loc.latitude, loc.longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### cluster 0 - Zoo Cluster

cluster 0 is shown as the red dots on the map above. It stands out as a single cluster since it is quite different than the rest. it most common venue is zoo exhibit since it is right next to the urban park and zoo. Following this we have mostly various restaruants.This cluster is much more uniform in its type of venues than the others.

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Gas Station,Caribbean Restaurant,Bank,Supermarket,Chinese Restaurant


#### cluster 1 - Downtown Cluster

The Downtown cluster is shown as purple on the map above. It is characterizes heavily by caual dining with coffee chops being by far the most common venue. Following this is a large assortment of restaurants that look to be upscale, followed by various nightlife such as cocktail bars and pubs. THere is the a minor assortment of hotels and commercial retial as well as a few attraction type establishments rounding out the top 10. All of these are what you would expect in a typical downtown setting.

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Park,Pub,Thai Restaurant,Farmers Market,Gastropub,Food Truck,Performing Arts Venue
3,North York,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Restaurant,Sandwich Place,Dessert Shop,Vietnamese Restaurant,Food Court,Cosmetics Shop
4,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Park,Ramen Restaurant,Gastropub,Thai Restaurant,Pizza Place,Restaurant,Italian Restaurant
7,North York,1,Coffee Shop,Restaurant,Japanese Restaurant,Bank,Pizza Place,Italian Restaurant,Middle Eastern Restaurant,Park,Sandwich Place,Gym
9,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Café,Restaurant,American Restaurant,Theater,Diner,Pizza Place
13,North York,1,Coffee Shop,Restaurant,Japanese Restaurant,Bank,Pizza Place,Italian Restaurant,Middle Eastern Restaurant,Park,Sandwich Place,Gym
15,Downtown Toronto,1,Coffee Shop,Café,Beer Bar,Restaurant,Japanese Restaurant,Gym,Farmers Market,Pizza Place,Gastropub,Hotel
16,York,1,Coffee Shop,Italian Restaurant,Bank,Pizza Place,Café,Grocery Store,Indian Restaurant,Caribbean Restaurant,Sandwich Place,Restaurant
19,East Toronto,1,Coffee Shop,Pub,Grocery Store,Breakfast Spot,Japanese Restaurant,Beach,Sandwich Place,Bakery,BBQ Joint,Ice Cream Shop
20,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Park,Beer Bar,Pizza Place,Farmers Market,Italian Restaurant


#### cluster 2 - Suburb Cluster

The suburb cluster is shown as light blue on the map above. This cluster is characterized by typical suburban venues with parks and coffee shops being the most common. The next most common venues consists of restaurants, stores, and markets/gas stations. This looks to be the makeup of your typical suburban neighborhood.

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,2,Pharmacy,Pizza Place,Gym / Fitness Center,Park,Fast Food Restaurant,Athletics & Sports,Intersection,Brewery,Gastropub,Mexican Restaurant
11,Etobicoke,2,Park,Pizza Place,Intersection,Farmers Market,Sandwich Place,Bank,Bakery,Fish & Chips Shop,Restaurant,Grocery Store
12,Scarborough,2,Park,Hotel,Neighborhood,Breakfast Spot,Grocery Store,Gym,Gym / Fitness Center,Italian Restaurant,Burger Joint,Fast Food Restaurant
18,Scarborough,2,Pizza Place,Fast Food Restaurant,Breakfast Spot,Train Station,Restaurant,Bank,Pharmacy,Beer Store,Discount Store,Juice Bar
28,North York,2,Park,Pizza Place,Coffee Shop,Gas Station,Bank,Bridal Shop,Men's Store,Mediterranean Restaurant,Shopping Mall,Supermarket
39,North York,2,Gas Station,Park,Trail,Bank,Restaurant,Shopping Mall,Café,Chinese Restaurant,Grocery Store,Athletics & Sports
40,North York,2,Coffee Shop,Vietnamese Restaurant,Pizza Place,Gas Station,Gym / Fitness Center,Athletics & Sports,Grocery Store,Bank,Pharmacy,Fast Food Restaurant
46,North York,2,Coffee Shop,Vietnamese Restaurant,Pizza Place,Gas Station,Gym / Fitness Center,Athletics & Sports,Grocery Store,Bank,Pharmacy,Fast Food Restaurant
50,North York,2,Park,Asian Restaurant,Sports Bar,Pharmacy,Coffee Shop,Electronics Store,Shopping Mall,Skating Rink,Mexican Restaurant,Latin American Restaurant
51,Scarborough,2,Park,Harbor / Marina,Pharmacy,Pub,Grocery Store,Discount Store,Beach,Coffee Shop,Sandwich Place,Ice Cream Shop


#### cluster 3 -  The Everything cluster.

Cluster 3 is shown in light green on the map above and consists of a single cluster. This cluster is kind of a mix between the downtown and suburban clusters. It's most common venue is coffee shops liek the downtown clsuter but it lacks the amount of parks the suburban cluster has. The next most common are again restaurants, stores, and markets. 

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Pharmacy,Intersection,Coffee Shop,Bank,Supermarket,Gas Station,Park,Medical Supply Store,Caribbean Restaurant,Office
1,North York,3,Coffee Shop,Middle Eastern Restaurant,Fast Food Restaurant,Gym,Park,Shoe Store,Gym / Fitness Center,Grocery Store,History Museum,Bank
5,Etobicoke,3,Pharmacy,Grocery Store,Bank,Playground,Supermarket,Golf Course,Café,Liquor Store,Shopping Mall,Skating Rink
10,North York,3,Coffee Shop,Bank,Grocery Store,Fast Food Restaurant,Restaurant,Gas Station,Fried Chicken Joint,Sandwich Place,Caribbean Restaurant,Pizza Place
14,East York,3,Coffee Shop,Park,Pizza Place,Sandwich Place,Bank,Pharmacy,Café,Gastropub,Bakery,Bar
17,Etobicoke,3,Baseball Field,Coffee Shop,Grocery Store,Café,Bank,Pizza Place,Convenience Store,Park,Flower Shop,Gas Station
22,Scarborough,3,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Bank,Fast Food Restaurant,Park,Bus Station,Department Store,Juice Bar
27,North York,3,Chinese Restaurant,Coffee Shop,Park,Bank,Pharmacy,Sandwich Place,Grocery Store,Bakery,Pizza Place,Cantonese Restaurant
32,Scarborough,3,Sandwich Place,Pharmacy,Pizza Place,Ice Cream Shop,Bank,Coffee Shop,Restaurant,Optical Shop,Gas Station,Beer Store
33,North York,3,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Park,Sandwich Place,Restaurant,Chinese Restaurant,Gas Station,Juice Bar,Bakery


#### cluster 4 -  Rural cluster

Cluster 4 is shown in orange on the map above. This cluster is unique like cluster 0 in that it consists of a single neighbourhood. What is interesting about this cluster is that its 2 most common venues are a national park and a farm as it is situated right in the Rogue National Park. The rest of the top 10 is almost exclusively restaurants. This seems to be typical of an outer city/ rural area.

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,4,National Park,Farm,Fast Food Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
